In [2]:
#                  IMPLEMENTATION OF MULTI-OBJECTIVE PARTICLE SWARM OPTIMIZATION
#                  -------------------------------------------------------------                  
#
#
## This is the Main Function to execute the MOPSO Process


import nbimporter
import random
from math import *
import numpy
import copy
import init_VELOCITY as initvelo
import init_PARTICLE_VALUES as initparval
import PARTICLE_FITNESS
import store_pBest
import init_NONDOMINATED_SOLUTION
import CROWDING
import UPDATE_VELOCITY_POSITION
import UPDATE_ARCHIVE
import UPDATE_pBest
import sensitivity_specificity_accuracy_fscore_AUC
reload(sensitivity_specificity_accuracy_fscore_AUC)
reload(UPDATE_pBest)
reload(UPDATE_ARCHIVE)
reload(UPDATE_VELOCITY_POSITION)
reload(CROWDING)
reload(init_NONDOMINATED_SOLUTION)
reload(store_pBest)
reload(PARTICLE_FITNESS)
reload(initvelo)
reload(initparval)


# main function

if __name__=='__main__':
    normal_count=0         # Counts the number of Normal samples to be considered for individual gene
    tumor_count=0          # Counts the number of Tumor samples to be considered for individual gene
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/PSO1/datasets/child.txt") as fp:
        for line in fp:
            if(line.startswith("normal")):
                normal_count+=1
            elif(line.startswith("tumor")):
                tumor_count+=1
       
    COUNT=normal_count+tumor_count        # total number of samples for each gene
    SWARM_SIZE=25                         # total number of particles in the population
    no_of_obj_func=3                      # total number of objective functions considered
    Iteration=100
    archive_size=25                       # set maximum capacity of archive
    no_of_nondom_sol=0                    # Initial number of nondominated solutions in archive
    
    PARTICLE_VAL=initparval.initialise_PARTICLE_VALUES(COUNT,SWARM_SIZE) # initialise position values of particles in population 
    PARTICLE_FIT=[[[] for j in range(no_of_obj_func)] for i in range(SWARM_SIZE)]  # fitness values of particles in population
    
    ARCHIVE_VAL=[[[]for j in range(COUNT)] for i in range(archive_size)]  #position values of particles in archive
    ARCHIVE_FIT=[[[]for j in range(no_of_obj_func)]for i in range(archive_size)]  #fitness values of particles in archive
    
    VELOCITY=initvelo.initialise_velocity(COUNT,SWARM_SIZE)  # initialise velocity of particle in population
    
    pBestPosition=[[[]for j in range(COUNT)]for i in range(SWARM_SIZE)] # personal best position for each particle in population
    pBestFitness=[[[]for j in range(no_of_obj_func)]for i in range(SWARM_SIZE)]  # personal fitness for each particle in population
    
#     PARTICLE_FITNESS.update_particle(PARTICLE_FIT,no_of_obj_func,SWARM_SIZE) # update fitness values of each particle in population
    
    store_pBest.p_Best(pBestPosition,pBestFitness,PARTICLE_VAL,PARTICLE_FIT,SWARM_SIZE,no_of_obj_func,COUNT) # Store initial personal bests (both variable and fitness values) of particles
   
    
    # the output is written in the file MOPSO_OUTPUT.txt
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/MOPSO_OUTPUT.txt","w+") as am:
        for t in range(Iteration):
            # Compute new fitness values for each particle in the population
            PARTICLE_FITNESS.update_particle(PARTICLE_FIT,no_of_obj_func,SWARM_SIZE)
            # insert nondominated particles in archive from population
            no_of_nondom_sol=init_NONDOMINATED_SOLUTION.initialise_nondom_sol(no_of_nondom_sol,SWARM_SIZE,ARCHIVE_VAL,ARCHIVE_FIT,PARTICLE_VAL,PARTICLE_FIT,no_of_obj_func,archive_size,COUNT)
            
            if(no_of_nondom_sol>2):
                no_of_nondom_sol=CROWDING.crowding_distance(no_of_nondom_sol,no_of_obj_func,COUNT,ARCHIVE_FIT,ARCHIVE_VAL)
            # Compute new velocity and position of each particle in the population
            no_of_nondom_sol=UPDATE_VELOCITY_POSITION.update_velocity_position(no_of_nondom_sol,SWARM_SIZE,COUNT,VELOCITY,pBestPosition,PARTICLE_VAL,ARCHIVE_VAL)

            # insert new non dominated particles in archive from population
            no_of_nondom_sol=UPDATE_ARCHIVE.update_archive(SWARM_SIZE,no_of_nondom_sol,archive_size,COUNT,ARCHIVE_VAL,PARTICLE_VAL,ARCHIVE_FIT,PARTICLE_FIT,no_of_obj_func)
            
            # update personal best for each particle in population
            UPDATE_pBest.update_pBest(SWARM_SIZE,no_of_obj_func,PARTICLE_FIT,pBestFitness,COUNT,pBestPosition,PARTICLE_VAL)
            
            NONDOM_PARTICLE=[int(0) for i in range(no_of_nondom_sol)]
            
            print (str(t))
            am.write("SIZE OF PARETO SET : ")
            am.write(str(no_of_nondom_sol))
            am.write("\n\n")
            am.write("ELEMENTS OF PARETO SET : ")
            for  i in range(no_of_nondom_sol):
                for j in range(SWARM_SIZE):
                    if(ARCHIVE_FIT[i][0]==PARTICLE_FIT[j][0]):
                        NONDOM_PARTICLE[i]=j
                        am.write(str(j+1))
                am.write("\t")
            am.write("\n\n")
                    
            am.write("FITNESS VALUES OF PARTICLES IN THE ARCHIVE : ")
            am.write("\n\n")
            am.write("WEIGHTED T SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][0]))
                am.write("\t\t")
            am.write("\n\n")
            am.write("WEIGHTED Z SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][1]))
                am.write("\t\t")
            am.write("\n\n")
            am.write("INTERACTION SCORE VALUES : ")
            for j in range(no_of_nondom_sol):
                am.write(str(ARCHIVE_FIT[j][2]))
                am.write("\t\t")
            am.write("\n")
            am.write("\n")
            am.write("**********************************************************************************************")    
            am.write("\n\n")
            
            temp_nondom=no_of_nondom_sol
            # reset number of nondominated particles, archive values and fitnesses to store values for next iteration
            no_of_nondom_sol=0
            
            ARCHIVE_VAL=[[[]for j in range(COUNT)] for i in range(archive_size)]
            ARCHIVE_FIT=[[[]for j in range(no_of_obj_func)]for i in range(archive_size)]
    
    # store the sensitivity , specificity , accuracy , f_score and area under curve value of the final non-dominated particles on archive
    VALUES_1=[[float(0.0) for j in range(5)] for i in range(temp_nondom)]
    with open("/home/sukanya/Desktop/PSO_and_MOPSO/MOPSO/Output.txt" , "w+") as sn:
        sensitivity_specificity_accuracy_fscore_AUC.values(SWARM_SIZE,NONDOM_PARTICLE,temp_nondom,VALUES_1) # calculation of the corresponding values

        sn.write("ELEMENTS OF PARETO SET : ")
        for i in range(temp_nondom):
            sn.write(str(NONDOM_PARTICLE[i]+1))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("SENSITIVITY SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][0]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("SPECIFICITY SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][1]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("ACCURACY MEASURE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][2]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("F SCORE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][3]))
            sn.write("\t")
        sn.write("\n\n")
        sn.write("AREA UNDER CURVE : ")
        for i in range(temp_nondom):
            sn.write(str(VALUES_1[i][4]))
            sn.write("\t")
        sn.write("\n\n")
            
    
            

Importing Jupyter notebook from sensitivity_specificity_accuracy_fscore_AUC.ipynb
Importing Jupyter notebook from UPDATE_pBest.ipynb
Importing Jupyter notebook from UPDATE_ARCHIVE.ipynb
Importing Jupyter notebook from UPDATE_VELOCITY_POSITION.ipynb
Importing Jupyter notebook from CROWDING.ipynb
Importing Jupyter notebook from init_NONDOMINATED_SOLUTION.ipynb
Importing Jupyter notebook from store_pBest.ipynb
Importing Jupyter notebook from PARTICLE_FITNESS.ipynb
Importing Jupyter notebook from init_VELOCITY.ipynb
Importing Jupyter notebook from init_PARTICLE_VALUES.ipynb
0
1
2
3
4
